In [1]:
#1 imports
import os
import math
import time
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import comb

np.random.seed(170786)

%matplotlib inline
#%matplotlib notebook
%load_ext autoreload
%autoreload 2
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

In [2]:
@jit
def wrap(r,L):
    if r[0] >= L:
        r[0] -= L
    elif r[0] < 0:
        r[0] += L
    if r[1] >= L:
        r[1] -= L
    elif r[1] < 0:
        r[1] += L
    if r[2] >= L:
        r[2] -= L
    elif r[2] < 0:
        r[2] += L
    return r

@jit
def r_ij(ri, rj, L):
    # min dist vector
    halfL = L/2.
    
    dx = rj[0]-ri[0]
    dy = rj[1]-ri[1]
    dz = rj[2]-ri[2]
    
    if dx >= halfL:
        dx -= L
    elif dx < -halfL:
        dx += L
    if dy >= halfL:
        dy -= L
    elif dy < -halfL:
        dy += L
    if dz >= halfL:
        dz -= L
    elif dz < -halfL:
        dz += L
    
    return np.array([dx, dy, dz])

#3 Minimum image distance
@jit
def dist(r1,r2,L):
    halfL = L/2.
    dx = r2[0]-r1[0]
    dy = r2[1]-r1[1]
    dz = r2[2]-r1[2]
    if dx >= halfL:
        dx -= L
    elif dx < -halfL:
        dx += L
    if dy >= halfL:
        dy -= L
    elif dy < -halfL:
        dy += L
    if dz >= halfL:
        dz -= L
    elif dz < -halfL:
        dz += L

    return math.sqrt(dx**2+dy**2+dz**2)

In [3]:
#4 Paiwise energy
@jit
def E_ij(s,sigma,epsilon):
    usub = sigma/s
    energy = 4*epsilon*(usub**12 - usub**6)
    return energy

#5 Pairwise force
@jit
def f_ij(r,sigma,epsilon):
    s = math.sqrt(np.sum(np.array(r)**2)) # r is rij
    usub = sigma/s
    f_mag = (-48*epsilon/s**2)*(usub**12 - 0.5*usub**6)
    return f_mag*r
#8 Total Energy
@jit
def E_i(r,i,xyz,L,sigma,epsilon):
    
    Ei_sum = 0
    for (j, particle) in enumerate(xyz):
        if j != i:
            s = dist(r, particle, L)
            Ei_sum += E_ij(s, sigma, epsilon)
    return Ei_sum
#System Energy and Pressure
@jit
def E_system(xyz,L,sigma,epsilon):
    N = xyz.shape[0]
    E = 0.
    for ii in range(N):
        r1 = xyz[ii,:]
        for jj in range(ii):
            r2 = xyz[jj,:]
            d = dist(r1,r2,L)
            E += E_ij(d,sigma,epsilon)
    return E

#6 Initialization - random
def rand_point(L):
    return L*(1 - np.random.random(3))

def init_rand(N,L,sigma):
    
    xyz = np.random.uniform(0,L,(N,3))

    for ii in range(N):
#         print('  Inserting particle %d' % (ii+1))
        xyz[ii,:] = np.random.uniform(0,L,3)
        r1 = xyz[ii,:]
        collision=1
        while(collision):
            collision=0
            for jj in range(ii):
                r2 = xyz[jj,:]
                d = dist(r1,r2,L)
                if d<sigma:
                    collision=1
                    break
            if collision:
                r1 = np.random.uniform(0,L,3)
                xyz[ii,:] = r1
#     print('Done!')

    # verifying all collisions resolved
    for ii in range(N):
        r1 = xyz[ii,:]
        for jj in range(ii):
            r2 = xyz[jj,:]
            d = dist(r1,r2,L)
            if d<sigma:
                raise Exception('Collision between particles %d and %d' % (ii+1,jj+1))
    
    return xyz

#7 Initialization - crystal
def init_xtal(N,L):
    
    xyz = np.zeros((N,3))
    
    K = int(np.ceil(N**(1/3))) # no. particles per side of cube

    counter = 0
    for ii in range(K):
        for jj in range(K):
            for kk in range(K):
                if counter<N:
                    idx = np.ravel_multi_index((ii,jj,kk), (K,K,K))
                    xyz[counter,:] = [ii/K*L,jj/K*L,kk/K*L]
                counter+=1
    
    return xyz

In [4]:
#9 Visualization
def plot_config(xyz, L):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter(xyz[:,0], xyz[:,1], xyz[:,2], zdir='z', s = 15, depthshade=True)
    
    ax.set_xlim([0,L])
    ax.set_ylim([0,L])
    ax.set_zlim([0,L])
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
    plt.draw()
    plt.show()
    
    return None

#10 Coordinate write
def xyzWrite(xyz,outfile,writestyle):
    
    nm_to_a = 10
    symbol = 'Ar'
    
    with open(outfile, writestyle) as xyz_file:
        # header
        xyz_file.write("%d\n%s\n" % (xyz.shape[0], ''))
        #lines
        for atom in xyz:
            (x, y, z) = nm_to_a*atom
            xyz_file.write("{:4} {:11.6f} {:11.6f} {:11.6f}\n".format(symbol, x, y, z))
    
    return None

In [5]:
@jit
def frij(s,sigma,epsilon):
    usub = sigma/s
    fr_mag = (-48*epsilon)*(usub**12 - 0.5*usub**6)
    return fr_mag

@jit
def pressure(N,L,T,xyz,sigma,epsilon):
    kB = 1.38064852e-2 #zJ/K
    V = L**3
    ideal = (N/V)*kB*T
    
    virial = 0.
    for ii in range(N):
        r1 = xyz[ii,:]
        for jj in range(ii):
            r2 = xyz[jj,:]
            sij = dist(r1,r2,L)
            virial += frij(sij,sigma,epsilon)
            
    press = ideal - 1/(3*V)*(virial)
    
    return press

In [6]:
# Metropolis 
def metropolis(log_chi):

    rand_val = np.random.random()
    if log_chi == 1:
        metropolis_accept = True
    elif rand_val == 0:
        metropolis_accept = False
    else:
        metropolis_accept = np.log(rand_val) < log_chi

    return metropolis_accept

@jit
def trial_disp(xyz, L, dispSize, N, sigma, epsilon, beta):
    
    rand_index = np.random.choice(range(N))
    rand_trans = dispSize*np.random.uniform(low=-1.0, high=1.0, size=3)
    r_old = xyz[rand_index]
    trial_r = wrap(r_old+rand_trans, L)
    
    Ejj_old = E_i( r_old ,rand_index,xyz,L,sigma,epsilon)
    Ejj_new = E_i( trial_r ,rand_index,xyz,L,sigma,epsilon)
    deltaE = Ejj_new - Ejj_old
    
    log_chi = min(0, -beta*deltaE)
    
    return rand_index, trial_r, deltaE, log_chi

@jit
def trial_volume(xyz, L_old, e_old, p_target, beta, N, sigma, epsilon):
    
    V_old = L_old**3
    dV=0.05*V_old
    deltaV = dV*np.random.uniform(low=-1.0, high=1.0)
    V_new = V_old + deltaV
    L_new = (V_new)**(1/3)
    scaler = L_new/L_old
    scaled_xyz = scaler*xyz
    deltaE = E_system(scaled_xyz, L_new, sigma, epsilon) - e_old
    log_chi = min(0, -beta*(deltaE + p_target*(deltaV))+N*np.log(V_new/V_old))
    
    return L_new, scaled_xyz, deltaE, log_chi


In [7]:
class ensemble_params:
    def __init__(self,T=298,N=40,L=2.5,itype='xtal',sigma=0.34,epsilon=1.65,dispSize=0.1,nSweeps=5000,printModulus=10,writeModulus=10):
        self.T=T #K (#T=298)
        self.N=N #(# N = 40)
        self.L=L #nm (# L = 2.5)
        
        self.sigma=sigma #nm (#sigma = 0.34)
        self.epsilon=epsilon #zJ (# epsilon = 1.65)
        self.dispSize=dispSize #nm (# dispSize = 0.1)
        kB = 1.38064852e-2 #zJ/K
        self.beta = 1/(T*kB)
        
        self.nSweeps=nSweeps #(# nSweeps = 5000) 
        self.printModulus=printModulus #sweeps (# printModulus = 10)
        self.writeModulus=writeModulus #sweeps (writeModulus = 10)
        

        if itype == 'xtal':
            self.xyz = init_xtal(N, L)
        elif itype == 'rand':
            self.xyz = init_rand(N,L,sigma)

In [8]:
def MCMC_trajectory(file, X, var_disp = True, atarget=.5, NPT_sim=False, p_target=30):
    xyz = np.copy(X.xyz)
    xyzWrite(xyz, file, 'w')
    
    # Trackables
    P_sweep = []
    E_sweep=[]
    xyz_traj=[]
    if NPT_sim:
        L_sweep = []
    tot_acpts = 0
    
    e_ii = E_system(xyz,X.L,X.sigma,X.epsilon)
    L = X.L
    dS = X.dispSize
    
    # Simulation
    for ii in range(1, X.nSweeps+1):
        ###
        swp_acpts=0
        # Displacement
        for jj in range(X.N):
            (index, r_new, deltaE, log_chi) = trial_disp(xyz, L, dS, X.N, X.sigma, X.epsilon, X.beta)
            # Update position: new position, energy, acceptance ratio
            if metropolis(log_chi):
                xyz[index] = r_new
                e_ii += deltaE
                tot_acpts+=1 
                swp_acpts +=1
        
        # Volume
        if NPT_sim:
            (L_new, scaled_xyz, deltaE, log_chi) = trial_volume(xyz, L, e_ii, p_target, X.beta, X.N, X.sigma, X.epsilon)
            if metropolis(log_chi):
                xyz=scaled_xyz
                e_ii+=deltaE
                L=L_new
#                 tot_acpts+=1       
        ###
        
        # Post-sweep observations
        p_ii = pressure(X.N,L,X.T,xyz,X.sigma,X.epsilon)
        ar_tot = tot_acpts/(ii*(X.N))
        ar_swp = swp_acpts/(X.N)
        if ii % X.printModulus == 0:
#             print("Sweep %4d: (AR = %.3f, dS = %.4f,  E = %8.4f zJ)" % (ii, ar_tot, dS, e_ii,))
            None
        if ii % X.writeModulus == 0:
            xyzWrite(xyz, file, 'a')
            if NPT_sim:
                L_sweep.append(L)
        # Update trackables
        if var_disp:
            correction = (ar_swp-atarget)
            if correction > 0:
                dS*=1.025
            elif correction < 0:
                dS*=0.975
        xyz_traj.append(xyz)
        P_sweep.append(p_ii)
        E_sweep.append(e_ii)
        
    
    
    if NPT_sim:
        sol = (np.array(xyz_traj), np.array(E_sweep), np.array(P_sweep), np.array(L_sweep))
    else:
        sol = (np.array(xyz_traj), np.array(E_sweep), np.array(P_sweep))
    
    return sol

In [14]:
# g_test1 = ensemble_params(N=50, T=298, nSweeps=800, itype='rand', dispSize=0.1)
liq_Ar_params = ensemble_params(N=125, T=85, nSweeps=1000, itype='rand', dispSize=0.1)

# g_test1_sol = MCMC_trajectory('gtest1.xyz', g_test1, NPT_sim=True, p_target=5, atarget=0.8)
rt_0 = time.time()
liq_Ar_sol = MCMC_trajectory('Ar_liq_MC.xyz', liq_Ar_params, NPT_sim=True, p_target=10, var_disp=False)
rt_f = time.time()
print('runtime = %f' % (rt_f-rt_0))

runtime = 4.111644


In [16]:
traj, Es, Ps, Ls = liq_Ar_sol

print(Ls)

[2.35256775 2.29073127 2.18829343 2.11864596 2.06680909 2.04974418
 2.01616739 1.98185927 1.94088118 1.92253743 1.90887376 1.91528815
 1.89824706 1.88437256 1.91310228 1.87222012 1.88668835 1.87887617
 1.87564441 1.87885715 1.83939654 1.82831102 1.83595319 1.8408789
 1.83245414 1.83825717 1.83464098 1.83832322 1.82732627 1.83883966
 1.84295693 1.85016722 1.85000764 1.85196374 1.83936563 1.83666743
 1.8323653  1.81955109 1.82256624 1.8145722  1.81473886 1.8085208
 1.80421093 1.80300056 1.82015139 1.81822719 1.81666049 1.82717098
 1.82477328 1.82914594 1.814161   1.814161   1.81985076 1.81985076
 1.82849219 1.80925875 1.81298525 1.83220051 1.81209462 1.81711937
 1.81059333 1.80867483 1.80907934 1.80907934 1.81430902 1.80231243
 1.80438027 1.81383788 1.80316065 1.79644912 1.81668519 1.80763621
 1.80654518 1.80079132 1.8056709  1.82370377 1.81776947 1.80253571
 1.80110078 1.80328942 1.80200234 1.80200234 1.80999827 1.81123345
 1.81422873 1.81590273 1.82508102 1.82387046 1.82609966 1.839526

In [18]:
# g_test1 = ensemble_params(N=50, T=298, nSweeps=800, itype='rand', dispSize=0.1)
liq_Ar_params = ensemble_params(N=200, T=85, nSweeps=1000, itype='rand', dispSize=0.1)

# g_test1_sol = MCMC_trajectory('gtest1.xyz', g_test1, NPT_sim=True, p_target=5, atarget=0.8)
rt_0 = time.time()
liq_Ar_sol2 = MCMC_trajectory('Ar_liq_MC2.xyz', liq_Ar_params, NPT_sim=True, p_target=20, var_disp=False)
rt_f = time.time()
print('runtime = %f' % (rt_f-rt_0))

runtime = 7.836978


In [20]:
traj, Es, Ps, Ls = liq_Ar_sol2

print(Ls[74])

2.10902950863094
